# 🎬 AI Video Dubber — Hindi Dubbing Pipeline

**One-click Kannada → Hindi video dubbing** with voice cloning, lip-sync, and face restoration.

### Setup
1. **Runtime → Change runtime type → T4 GPU**
2. Run all cells in order
3. Find your output in `outputs/final_dubbed.mp4`

---

## Step 0: GPU Check & Environment Setup

In [ ]:
# Verify GPU is available
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU detected! Go to Runtime → Change runtime type → T4 GPU")

## Step 1: Clone Repository & Install Dependencies

In [ ]:
%%time
# Clone the AI Video Dubber repository
!git clone https://github.com/grafenx07/AI-video-dubber.git 2>/dev/null || echo "Already cloned"
%cd AI-video-dubber

# Check Python version
import sys
print(f"Python: {sys.version}")

# Install core dependencies (without TTS first)
!pip install -q openai-whisper deep-translator edge-tts nest-asyncio \
    gfpgan librosa soundfile pydub tqdm opencv-python-headless

# ═══════════════════════════════════════════════════════════════
# FIX: Coqui TTS requires Python <3.12
# Install coqui-tts fork that supports newer Python versions
# ═══════════════════════════════════════════════════════════════
if sys.version_info >= (3, 12):
    print("\n⚠️ Python 3.12+ detected — installing coqui-tts fork...")
    import subprocess
    r = subprocess.run([sys.executable, "-m", "pip", "install", "-q", "coqui-tts"],
                       capture_output=True, text=True)
    if r.returncode != 0:
        r = subprocess.run([sys.executable, "-m", "pip", "install", "-q",
                           "git+https://github.com/idiap/coqui-ai-TTS.git"],
                          capture_output=True, text=True)
    if r.returncode != 0:
        print("⚠️ TTS install failed — will use Edge TTS fallback (still good quality)")
else:
    !pip install -q TTS

# Install IndicTrans2 toolkit (optional — for better translation)
!pip install -q IndicTransToolkit 2>/dev/null || echo "IndicTrans2 not available, using Google Translate"

# Verify TTS availability
try:
    from TTS.api import TTS
    print("\n✅ Coqui TTS installed — XTTS voice cloning available!")
except ImportError:
    print("\n⚠️ Coqui TTS not available — will use Edge TTS (still high quality Hindi)")

print("\n✅ All dependencies installed!")

## Step 2: Setup Wav2Lip (Lip Synchronization)

In [ ]:
%%time
import os
import subprocess

# Clone Wav2Lip
if not os.path.exists("Wav2Lip"):
    !git clone https://github.com/Rudrabha/Wav2Lip.git
    !cd Wav2Lip && pip install -q -r requirements.txt 2>/dev/null
else:
    print("Wav2Lip already cloned")

# ═══════════════════════════════════════════════════
# CRITICAL FIX: Patch Wav2Lip for numpy >= 1.24
# np.float was removed in numpy 1.24; Wav2Lip uses it
# ═══════════════════════════════════════════════════
wav2lip_audio = "Wav2Lip/audio.py"
if os.path.exists(wav2lip_audio):
    with open(wav2lip_audio, "r") as f:
        content = f.read()
    if "np.float)" in content or "np.float," in content:
        content = content.replace("np.float)", "np.float64)")
        content = content.replace("np.float,", "np.float64,")
        with open(wav2lip_audio, "w") as f:
            f.write(content)
        print("✅ Patched Wav2Lip audio.py for numpy compatibility")

# Also patch models/wav2lip.py if needed
for fpath in ["Wav2Lip/models/wav2lip.py", "Wav2Lip/models/wav2lip_gan.py"]:
    if os.path.exists(fpath):
        with open(fpath, "r") as f:
            content = f.read()
        patched = content.replace("np.float)", "np.float64)")
        if patched != content:
            with open(fpath, "w") as f:
                f.write(patched)
            print(f"✅ Patched {fpath}")

# Download Wav2Lip GAN model (higher quality)
os.makedirs("Wav2Lip/checkpoints", exist_ok=True)
if not os.path.exists("Wav2Lip/checkpoints/wav2lip_gan.pth"):
    print("Downloading wav2lip_gan.pth...")
    !pip install -q gdown
    # Try primary mirror
    !gdown --id 1kMFGEASqoGA__fxJ0Z0mEKLpCyF52FK- -O Wav2Lip/checkpoints/wav2lip_gan.pth 2>/dev/null \
        || echo "Primary mirror failed, trying alternate..."
    # Verify file was downloaded (not an error page)
    if os.path.exists("Wav2Lip/checkpoints/wav2lip_gan.pth"):
        size = os.path.getsize("Wav2Lip/checkpoints/wav2lip_gan.pth")
        if size < 100_000_000:  # Model should be ~400MB+
            os.remove("Wav2Lip/checkpoints/wav2lip_gan.pth")
            print("⚠️ Downloaded file too small, re-downloading...")
            !gdown "https://drive.google.com/uc?id=1kMFGEASqoGA__fxJ0Z0mEKLpCyF52FK-" -O Wav2Lip/checkpoints/wav2lip_gan.pth \
                || echo "⚠️ Auto-download failed. Please download manually from README."
        else:
            print(f"✅ wav2lip_gan.pth downloaded ({size / 1e6:.0f} MB)")
else:
    print("wav2lip_gan.pth already exists")

# Download face detection model
face_det_dir = "Wav2Lip/face_detection/detection/sfd"
os.makedirs(face_det_dir, exist_ok=True)
if not os.path.exists(f"{face_det_dir}/s3fd.pth"):
    !wget -q -O {face_det_dir}/s3fd.pth https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth
    print("✅ s3fd.pth downloaded")

print("\n✅ Wav2Lip setup complete!")

## Step 3: Download Source Video

In [ ]:
%%time
import os

# Download the Supernan training video from Google Drive
VIDEO_FILE = "supernan_training.mp4"

if not os.path.exists(VIDEO_FILE):
    !pip install -q gdown
    # Google Drive file ID from shared link
    !gdown --id 1uDzLVEow_gAJsXnNjbSoskzVLZ4d7opW -O {VIDEO_FILE}
    print(f"\n✅ Video downloaded: {VIDEO_FILE}")
else:
    print(f"Video already exists: {VIDEO_FILE}")

# Show video info
!ffprobe -v quiet -print_format json -show_format -show_streams {VIDEO_FILE} | python -m json.tool | head -30

## Step 4: Run the Dubbing Pipeline

This runs the complete pipeline:
1. Extract 15-second clip (0:15 - 0:30)
2. Transcribe Kannada speech (Whisper)
3. Translate to Hindi (Google Translate / IndicTrans2)
4. Generate Hindi speech with voice cloning (XTTS v2)
5. Align audio duration to video
6. Lip-sync video (Wav2Lip)
7. Enhance face quality (GFPGAN)

**Expected time: ~5-10 minutes on T4 GPU**

In [ ]:
%%time

# ════════════════════════════════════════════
#  CONFIGURATION — Adjust these as needed!
# ════════════════════════════════════════════
INPUT_VIDEO = "supernan_training.mp4"
START_TIME = 15      # Start at 0:15
END_TIME = 30        # End at 0:30
WHISPER_MODEL = "medium"   # Use 'medium' for better Kannada accuracy (small often fails)
TRANSLATION = "google"     # google|indictrans2|seamless
TTS_METHOD = "xtts"        # xtts|edge (auto-falls back to edge if needed)
SPEECH_RATE = 1.05         # Slightly faster for Hindi

# Run the pipeline
!python dub_video.py \
    --input {INPUT_VIDEO} \
    --start {START_TIME} \
    --end {END_TIME} \
    --whisper-model {WHISPER_MODEL} \
    --translation {TRANSLATION} \
    --tts {TTS_METHOD} \
    --speech-rate {SPEECH_RATE} \
    --wav2lip-batch-size 8

print("\n" + "="*60)
print("✅ Pipeline complete! Check outputs/final_dubbed.mp4")
print("="*60)

## Step 5: Preview Results

In [ ]:
# Display the output video inline
from IPython.display import HTML, display
from base64 import b64encode
import os

output_video = "outputs/final_dubbed.mp4"

if os.path.exists(output_video):
    # Show video inline
    video_data = open(output_video, "rb").read()
    b64 = b64encode(video_data).decode()
    display(HTML(f"""
    <h3>🎬 Final Dubbed Video</h3>
    <video width="640" controls>
        <source src="data:video/mp4;base64,{b64}" type="video/mp4">
    </video>
    """))

    # Show file info
    size_mb = os.path.getsize(output_video) / (1024 * 1024)
    print(f"\nFile: {output_video}")
    print(f"Size: {size_mb:.1f} MB")
else:
    print("❌ Output video not found. Check pipeline logs for errors.")

## Step 6: Review Intermediate Outputs

In [ ]:
import json
import os

# Show transcription
transcription_file = "outputs/03_transcription.json"
if os.path.exists(transcription_file):
    with open(transcription_file, "r") as f:
        data = json.load(f)
    print("📝 Kannada Transcription:")
    print(f"   {data['text']}")
    print(f"   Segments: {len(data.get('segments', []))}")
    print()

# Show translation
translation_file = "outputs/04_translation.json"
if os.path.exists(translation_file):
    with open(translation_file, "r") as f:
        data = json.load(f)
    print("🌍 Hindi Translation:")
    print(f"   {data['full_text']}")
    print()
    if data.get('segments'):
        print("   Segment-by-segment:")
        for seg in data['segments']:
            print(f"   [{seg['start']:.1f}s-{seg['end']:.1f}s] {seg['original']}")
            print(f"   → {seg['translated']}")
            print()

In [ ]:
# Play the Hindi audio
from IPython.display import Audio, display
import os

for audio_file, label in [
    ("outputs/02_audio.wav", "Original Kannada Audio"),
    ("outputs/05_hindi_raw.wav", "Generated Hindi Audio (raw)"),
    ("outputs/06_hindi_aligned.wav", "Hindi Audio (aligned to video)"),
]:
    if os.path.exists(audio_file):
        print(f"\n🔊 {label}:")
        display(Audio(audio_file))

## Step 7: Download Output

In [ ]:
# Download the final video
from google.colab import files
import os

output_video = "outputs/final_dubbed.mp4"
if os.path.exists(output_video):
    files.download(output_video)
    print("✅ Download started!")
else:
    print("❌ Output not found")

---

## 🔧 Troubleshooting

| Issue | Solution |
|-------|----------|
| `CUDA out of memory` | Reduce `--wav2lip-batch-size` to 4, use `--whisper-model base` |
| `Wav2Lip model not found` | Re-run Step 2 or download manually |
| `No GPU available` | Runtime → Change runtime type → T4 GPU |
| `Edge TTS timeout` | Retry or switch to `--tts xtts` |
| `Translation is too literal` | Use `--translation indictrans2` for better Hindi |

## 📊 Quick Alternative: Skip Heavy Steps

If Wav2Lip or GFPGAN are causing issues, run without them for a quick result:

In [ ]:
# Quick mode: Audio-only dubbing (no lip-sync, no enhancement)
# Good for testing translation + voice cloning quality

!python dub_video.py \
    --input supernan_training.mp4 \
    --start 15 --end 30 \
    --skip-lipsync \
    --skip-enhancement \
    --output-dir outputs_quick